# Pandas in Practice — A Hands-On Notebook

This notebook is a practical, example-driven refresher on **pandas**.

You’ll learn:
- `Series` and `DataFrame`
- Reading data and quick inspection
- Selecting rows/columns with `[]`, `.loc`, `.iloc`
- Filtering, creating columns, and handling missing data
- Sorting
- `groupby` + aggregations
- Reshaping with `pivot_table` and `melt`
- Joining tables with `merge`
- Dates and resampling
- Saving outputs

**Tip:** Run cells top-to-bottom. Examples build on previous variables.

## 0. Setup

In [ ]:
import pandas as pd
import numpy as np
pd.__version__

## 1. Core Objects: Series and DataFrame

In [ ]:
s = pd.Series([10, 20, 30], index=['a', 'b', 'c'])
s

In [ ]:
df = pd.DataFrame({
    'name': ['Ada', 'Linus', 'Grace'],
    'age': [32, 54, 40],
    'role': ['admin', 'user', 'admin']
})
df

In [ ]:
df.dtypes

## 2. Quick Inspection

In [ ]:
df.head()

In [ ]:
df.shape, df.columns.tolist()

In [ ]:
df.info()

In [ ]:
df.describe(numeric_only=True)

## 3. Selecting Data

In [ ]:
# Columns
df['age']

In [ ]:
df[['name', 'age']]

In [ ]:
# Rows/cells by position
df.iloc[0]

In [ ]:
df.iloc[0:2]

In [ ]:
df.iloc[0, 1]  # row 0, col 1

In [ ]:
# Rows/cells by label using .loc (set an index first)
df2 = df.set_index('name')
df2.loc['Ada']

In [ ]:
df2.loc['Ada', 'age']

## 4. Filtering Rows (Boolean Indexing)

In [ ]:
df[df['role'] == 'admin']

In [ ]:
df[df['age'] >= 40]

In [ ]:
# Multiple conditions: use & and parentheses
df[(df['role'] == 'admin') & (df['age'] >= 35)]

In [ ]:
df[df['role'].isin(['admin', 'manager'])]

In [ ]:
df[df['name'].str.contains('a', case=False, na=False)]

## 5. Creating and Updating Columns

In [ ]:
df = df.copy()
df['age_plus_10'] = df['age'] + 10
df['is_admin'] = df['role'].eq('admin')
df

In [ ]:
df['tier'] = np.where(df['age'] >= 40, 'senior', 'junior')
df

In [ ]:
# .assign is nice for chaining
df_assigned = df.assign(
    age_group=pd.cut(df['age'], bins=[0, 35, 50, 120], labels=['young', 'mid', 'older'])
)
df_assigned

## 6. Missing Data (NaN): Detect, Drop, Fill

In [ ]:
messy = pd.DataFrame({
    'name': ['Ada', 'Linus', 'Grace', None],
    'age': [32, np.nan, 40, 28],
    'role': ['admin', 'user', None, 'user']
})
messy

In [ ]:
messy.isna().sum()

In [ ]:
# Drop rows with missing age
messy.dropna(subset=['age'])

In [ ]:
# Fill missing values
filled = messy.copy()
filled['age'] = filled['age'].fillna(filled['age'].median())
filled['role'] = filled['role'].fillna('unknown')
filled['name'] = filled['name'].fillna('missing')
filled

## 7. Sorting

In [ ]:
filled.sort_values('age')

In [ ]:
filled.sort_values(['role', 'age'], ascending=[True, False])

## 8. Groupby (Summaries Like Pivot Tables)

In [ ]:
sales = pd.DataFrame({
    'date': pd.to_datetime(['2026-01-01','2026-01-01','2026-01-02','2026-01-02','2026-01-03']),
    'store': ['A','B','A','B','A'],
    'item': ['apple','apple','banana','apple','apple'],
    'units': [10, 5, 7, 3, 4],
    'price': [1.0, 1.0, 2.0, 1.0, 1.0]
})
sales['revenue'] = sales['units'] * sales['price']
sales

In [ ]:
sales.groupby('store')['revenue'].sum()

In [ ]:
sales.groupby('store').agg(
    total_units=('units', 'sum'),
    total_revenue=('revenue', 'sum'),
    avg_units=('units', 'mean')
)

In [ ]:
sales.groupby(['store', 'item'])['revenue'].sum()

## 9. Reshaping: pivot_table and melt

In [ ]:
pivot = sales.pivot_table(index='date', columns='store', values='revenue', aggfunc='sum')
pivot

In [ ]:
long_again = pivot.reset_index().melt(id_vars='date', var_name='store', value_name='revenue')
long_again

## 10. Joining Tables (SQL-style merges)

In [ ]:
customers = pd.DataFrame({
    'customer_id': [1, 2, 3],
    'name': ['Ada', 'Linus', 'Grace']
})
orders = pd.DataFrame({
    'order_id': [101, 102, 103],
    'customer_id': [1, 1, 3],
    'total': [50.0, 20.0, 99.0]
})
orders.merge(customers, on='customer_id', how='left')

## 11. Dates and Resampling

In [ ]:
sales['day_name'] = sales['date'].dt.day_name()
sales[['date', 'day_name', 'revenue']]

In [ ]:
sales[sales['date'] >= '2026-01-02']

In [ ]:
daily_revenue = sales.set_index('date').resample('D')['revenue'].sum()
daily_revenue

## 12. Saving Output (CSV, Excel, Parquet)

In [ ]:
from pathlib import Path
out_dir = Path('pandas_out')
out_dir.mkdir(exist_ok=True)

# Save CSV
csv_path = out_dir / 'sales_clean.csv'
sales.to_csv(csv_path, index=False)
csv_path

In [ ]:
# Save Excel (requires openpyxl installed)
xlsx_path = out_dir / 'sales_report.xlsx'
sales.to_excel(xlsx_path, index=False)
xlsx_path

In [ ]:
# Save Parquet (requires pyarrow or fastparquet installed)
parquet_path = out_dir / 'sales.parquet'
try:
    sales.to_parquet(parquet_path, index=False)
    parquet_path
except Exception as e:
    f'Parquet not saved (missing dependency like pyarrow). Error: {e}'

## 13. Mini Exercises

Try these using the `sales` DataFrame:

1. Filter rows where `store == 'A'` and `units >= 5`.
2. Create a column `high_value` that is True if `revenue >= 10`.
3. Compute total revenue per `item` (groupby).
4. Create a pivot table of `units` with rows=`date`, columns=`item`.
5. Merge `orders` with `customers` and compute total spend per customer.


In [ ]:
# Sample solutions (clear and do them yourself if you want!)
q1 = sales[(sales['store'] == 'A') & (sales['units'] >= 5)]

q2 = sales.assign(high_value=sales['revenue'] >= 10)

q3 = sales.groupby('item')['revenue'].sum().sort_values(ascending=False)

q4 = sales.pivot_table(index='date', columns='item', values='units', aggfunc='sum')

merged = orders.merge(customers, on='customer_id', how='left')
q5 = merged.groupby('name')['total'].sum().sort_values(ascending=False)

q1, q2[['date','store','item','units','revenue','high_value']], q3, q4, q5

## 14. Key Takeaways

- Use `read_csv()` + `head()`/`info()` to quickly understand a dataset.
- Use `.loc` (labels) and `.iloc` (positions) intentionally.
- Prefer vectorized column operations over loops.
- `groupby` + `agg` is the main tool for summaries.
- `pivot_table` and `melt` convert between wide/long formats.
- `merge` is how you join tables like SQL.
